In [3]:
import urllib
import json
import numpy as np
import csv
import math
import h3
import plotly.express as px
import pandas as pd


Spb coordinates (square
)

In [14]:
lat_min = 59.804842
lat_max = 60.093279
lon_min = 30.094747
lon_max = 30.609159

In [15]:
def get_coords(subdivs, lat, lon):
	return { "lat": lat_min + (lat_max - lat_min) / subdivs * lat, "lon": lon_min + (lon_max - lon_min) / subdivs * lon }

In [16]:
def get_bounds(subdivs, lat_a, lon_a, lat_b, lon_b):
	a = get_coords(subdivs, lat_a, lon_a)
	b = get_coords(subdivs, lat_b, lon_b)
	return str(a["lon"]) + "," + str(a["lat"]) + "," + str(b["lon"]) + "," + str(b["lat"])

In [17]:
def load(cat, subdivs, lat_a, lon_a):
	lat_b = lat_a + 1
	lon_b = lon_a + 1
	bounds = get_bounds(subdivs, lat_a, lon_a, lat_b, lon_b)
	url = "https://spb.spravker.ru/api/map/?category=" + str(cat) + "&bounds=" + bounds + "&callback=id_170868754597686677116"
	data = urllib.request.urlopen(url).read().decode('utf-8')[25:-1]
	return json.loads(data)

Create list with all available categories, subcategories, id of subcategories and number of intities in subcategory

In [ ]:
def find_all_codes():
    import urllib.request

    f = urllib.request.urlopen("https://spb.spravker.ru/").read()
    f=str(f.decode('utf-8'))
    f=(f.split('<h3 class="category-widget__title">'))
    f=f[1:]
    cat_code=[]
    for index in f:
        cat_code.append(index.split('a href=')[1].split('title')[0].replace('"','').replace(' ',''))
    return cat_code

sub_cat_code=[]
cat_code=[]
number_of_items=[]
for cat_code_i in find_all_codes():
    f = urllib.request.urlopen("https://spb.spravker.ru"+cat_code_i).read()
    f=str(f.decode('utf-8'))
    f=(f.split('<li class="list-count__item">'))
    f=f[1:]
    for index in f:
        sub_cat_code.append(index.split('a href=')[1].split('title')[0].replace('"','').replace(' ',''))
        cat_code.append(cat_code_i)
        number_of_items.append(index.split('data-count="')[1].split('"')[0])


def find_cat_num(sub_cat_code_num):
	f = urllib.request.urlopen("https://spb.spravker.ru"+sub_cat_code_num).read()
	f=str(f.decode('utf-8'))
	return f.split('data-category=')[1].split('\n')[0].replace('"','')

sub_cat_num=[]
for index in sub_cat_code:
	cat_num = find_cat_num(index)
	sub_cat_num.append(cat_num)
	print(index, cat_num)
     

import pandas as pd
categories=pd.DataFrame(cat_code,columns=['cat_code'])
categories['sub_cat_code']=sub_cat_code
categories['sub_cat_code_num']=sub_cat_num
categories['sub_cat_code_count']=number_of_items
#categories.to_csv('C:/Users/Administrator/Desktop/categories.csv',sep=';',index=False)

Parcing

In [92]:
for cat in np.unique(categories['cat_code']):
	print("category", cat)
	data = []
	subcats = categories[categories['cat_code']==cat]["sub_cat_code_num"]
	for subcat in subcats:
		subname = categories[categories['sub_cat_code_num']==subcat]["sub_cat_code"].values[0]
		print("subcat", subname)
		count = categories[categories['sub_cat_code_num']==subcat]["sub_cat_code_count"].values[0]
		subdivs = math.ceil((float(count) / 80.0) ** 0.5)
		if subdivs == 0: subdivs = 1
		print("\t", subdivs, "subdivs")
		for lat in range(subdivs):
			for lon in range(subdivs):
				raw = load(subcat, subdivs, lat, lon)
				data += (list(map(lambda f: {
					'coordinates': f['geometry']['coordinates'],
					'name': f['properties']['balloonContentHeader'],
					'link': f['properties']['balloonContentFooter'],
					'category': cat,
					'subcategory': subname,
					'subcat_id': subcat
					}, raw["features"])))
			print("\t", len(data))
	

	with open(cat.replace("/","") + '.csv', 'w', encoding='UTF8', newline='') as f:
		header = ['coordinates', 'name', 'link', 'category', 'subcategory', 'subcat_id']
		w = csv.DictWriter(f, data[0].keys(), delimiter=';')
		w.writeheader()
		w.writerows(data)
	print("saved", len(data), "entries")


category /avto/
subcat /agzs/
	 3 subdivs
	 134
	 232
	 296
subcat /azs/
	 4 subdivs
	 575
	 849
	 1154
	 1268
subcat /abrazivnaia-polirovka-kuzova/
	 5 subdivs
	 1544
	 1908
	 2367
	 2800
	 2975
subcat /avariinoe-otkliuchenie-signalizatsii/
	 3 subdivs
	 3147
	 3384
	 3471
subcat /avtoaksessuary/
	 7 subdivs
	 3771
	 4344
	 4952
	 5675
	 6228
	 6553
	 6687
subcat /avtoakustika-1/
	 3 subdivs
	 6874
	 7137
	 7246
subcat /avtoatele/
	 4 subdivs
	 7495
	 7909
	 8344
	 8462
subcat /avtoauktsiony/
	 2 subdivs
	 8552
	 8611
subcat /avtodrom/
	 1 subdivs
	 8672
subcat /avtozapchasti/
	 11 subdivs
	 9257
	 9955
	 10912
	 11870
	 12478
	 13153
	 13840
	 14516
	 15034
	 15329
	 15509
subcat /avtozapchasti-audi/
	 4 subdivs
	 15741
	 16092
	 16337
	 16464
subcat /avtozapchasti-bmw/
	 4 subdivs
	 16685
	 17009
	 17237
	 17352
subcat /avtozapchasti-changfeng/
	 3 subdivs
	 17496
	 17603
	 17658
subcat /avtozapchasti-chery/
	 3 subdivs
	 17882
	 18064
	 18156
subcat /avtozapchasti-chevrolet/
	 4 su

Easier to open file

In [36]:
from os import listdir
from os.path import isfile, join
mypath='D:/Учеба/2 курс мага/диплом/geodata'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
first_file=pd.read_csv(mypath+'/'+onlyfiles[0],sep=';')
for files in onlyfiles[1:]:
    first_file=first_file.append(pd.read_csv(mypath+'/'+files,sep=';'))
    


In [37]:
first_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3326685 entries, 0 to 107569
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   coordinates  object
 1   name         object
 2   link         object
 3   category     object
 4   subcategory  object
 5   subcat_id    int64 
dtypes: int64(1), object(5)
memory usage: 177.7+ MB


In [38]:
new_cats=pd.read_csv('C:/Users/Asus/Downloads/Telegram Desktop/categories.csv',sep=';')

In [39]:
first_file=first_file.reset_index()



In [40]:
first_file.head(10)

,index,coordinates,name,link,category,subcategory,subcat_id
0,0,"[30.205252, 59.8616]",GPower,<a href='https://spb.spravker.ru/agzs/gpower-1...,/avto/,/agzs/,3
1,1,"[30.205848, 59.86133]",Global Gas,<a href='https://spb.spravker.ru/agzs/globalga...,/avto/,/agzs/,3
2,2,"[30.246048, 59.877287]",Газпром газомоторное топливо,<a href='https://spb.spravker.ru/org/gazprom-t...,/avto/,/agzs/,3
3,3,"[30.164843, 59.805513]",ВиГаз,<a href='https://spb.spravker.ru/agzs/vigaz.ht...,/avto/,/agzs/,3
4,4,"[30.168601, 59.811463]",РосГаз,<a href='https://spb.spravker.ru/org/rosgaz12....,/avto/,/agzs/,3
5,5,"[30.189314, 59.824641]",Vervex,<a href='https://spb.spravker.ru/org/vervex17....,/avto/,/agzs/,3
6,6,"[30.215474, 59.846405]",Северное,<a href='https://spb.spravker.ru/agzs/agzs-zao...,/avto/,/agzs/,3
7,7,"[30.202743, 59.857229]",СЗТК,<a href='https://spb.spravker.ru/agzs/sztk-1.h...,/avto/,/agzs/,3
8,8,"[30.191326, 59.825494]",СЗТК,<a href='https://spb.spravker.ru/agzs/sztk-8.h...,/avto/,/agzs/,3
9,9,"[30.219467, 59.860337]",АГЗС,<a href='https://spb.spravker.ru/agzs/agzs-4.h...,/avto/,/agzs/,3


Data pre-processing

In [41]:

first_file['link']=list(map(lambda x:x.split('=')[1].split('>')[0][1:-1],first_file['link']))
first_file['lat']=list(map(lambda x:float(x.split(',')[1][:-1]),first_file['coordinates']))
first_file['lon']=list(map(lambda x:float(x.split(',')[0][1:]),first_file['coordinates']))
first_file['h3_9']=list(map(lambda lat,lng:h3.latlng_to_cell(lat,lng,9),first_file['lat'],first_file['lon']))

In [42]:
new_cats.head(3)

,cat_code,sub_cat_code,sub_cat_code_num,new_sub_cat_code
0,/avto/,/agzs/,3,agzs or azs
1,/avto/,/azs/,2,agzs or azs
2,/avto/,/abrazivnaia-polirovka-kuzova/,676,avto_service


In [43]:
first_file=pd.merge(first_file,new_cats[['sub_cat_code_num','new_sub_cat_code']],how='left',right_on='sub_cat_code_num',left_on='subcat_id')
first_file.head(3)

,index,coordinates,name,link,category,subcategory,subcat_id,lat,lon,h3_9,sub_cat_code_num,new_sub_cat_code
0,0,"[30.205252, 59.8616]",GPower,https://spb.spravker.ru/agzs/gpower-1.htm,/avto/,/agzs/,3,59.861600,30.205252,8911061503bffff,3,agzs or azs
1,1,"[30.205848, 59.86133]",Global Gas,https://spb.spravker.ru/agzs/globalgas.htm,/avto/,/agzs/,3,59.861330,30.205848,8911061503bffff,3,agzs or azs
2,2,"[30.246048, 59.877287]",Газпром газомоторное топливо,https://spb.spravker.ru/org/gazprom-transgaz2.htm,/avto/,/agzs/,3,59.877287,30.246048,89110615ec7ffff,3,agzs or azs


In [44]:
first_file['new_sub_cat_code']=list(map(str,first_file['new_sub_cat_code']))

In [45]:
np.unique(first_file['new_sub_cat_code'])

array(['RE-service', 'additional education', 'agzs or azs', 'apteki',
       'avto_buy', 'avto_service', 'avto_shop', 'bezopasnost', 'biznes',
       'cemetery', 'clothers', 'clothes', 'dom-i-interer',
       'eda-i-napitki', 'furniture', 'gai-gibdd',
       'gosudarstvo-i-obschestvo', 'hobby', 'hospital', 'hospitality',
       'internet-service', 'kindergarten', 'kino', 'krasota-i-zdorove',
       'library', 'med-service', 'med-shop', 'museum', 'nan',
       'nochnye-kluby', 'otdyih-i-razvlecheniya', 'park',
       'pozharnye-chasti', 'punkt-vydachi', 'religion', 'rynki',
       'safety-shop', 'school', 'shop', 'sluzhby-ekstrennogo-vyzova',
       'sluzhby-spaseniia', 'social housing', 'sport', 'sport shop',
       'supermarket', 'tech', 'theater', 'torgovye-centry',
       'tourism services', 'tovaryi-dlya-detej', 'university',
       'upravlenie-obrazovaniem', 'vet clinics', 'vet service',
       'vet-shop'], dtype=object)

In [46]:



color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(first_file[first_file['new_sub_cat_code']=='park'], 
                        lat="lat", 
                        lon="lon", 
                        hover_name="name", 
                        hover_data=["name"],
                        #color="Listed",
                        #color_continuous_scale=color_scale,
                        #size="Listed",
                        zoom=8, 
                        height=800,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [47]:
first_file['count']=1
#pd.pivot_table(first_file,index='h3_9',columns='category',values='count',aggfunc=sum)
pd.pivot_table(first_file,index='h3_9',values='count',aggfunc=sum)

,count
h3_9,
89110600103ffff,39
89110600113ffff,903
89110600117ffff,7
89110600123ffff,12
89110600127ffff,15
...,...
891106b9e8bffff,6
891106b9ebbffff,12
891106b9ec3ffff,1


Add parks from 2gis

In [60]:
first_file[first_file['new_sub_cat_code']=='park'].head(3)

,index,coordinates,name,link,category,subcategory,subcat_id,lat,lon,h3_9,sub_cat_code_num,new_sub_cat_code,count
2582459,721,"[30.421268, 59.85265]",Дот оборонительного рубежа Ижора,https://spb.spravker.ru/muzei/dot-o-18-muzej-o...,/kultura-i-iskusstvo/,/botanicheskie-sady/,1034,59.852650,30.421268,89110600927ffff,1034,park,1
2582460,722,"[30.277181, 59.917022]",Адмиралтейские верфи,https://spb.spravker.ru/muzei/muzei-admiraltei...,/kultura-i-iskusstvo/,/botanicheskie-sady/,1034,59.917022,30.277181,8911061599bffff,1034,park,1
2582461,723,"[30.339204, 59.97724]",ФГБОУВО Санкт-Петербургский государственный пе...,https://spb.spravker.ru/org/muzej-spbgpmu.htm,/kultura-i-iskusstvo/,/botanicheskie-sady/,1034,59.977240,30.339204,891106a9287ffff,1034,park,1


In [61]:
parks=pd.read_csv('C:/Users/Asus/Downloads/Telegram Desktop/ПаркиКиО.csv',sep=',')
parks['lat']=parks['Широта']
parks['lon']=parks['Долгота']
parks['link']=parks['2GIS URL']
parks['category']='/kultura-i-iskusstvo/'
parks['subcategory']='/botanicheskie-sady/'
parks['new_sub_cat_code']='park'
parks['subcat_id']=1034
parks['h3_9']=list(map(lambda lat,lng:h3.latlng_to_cell(lat,lng,9),parks['lat'],parks['lon']))
parks['name']=parks['Наименование']

In [62]:
parks.columns

Index(['Наименование', 'Описание', 'Рубрики', 'Адрес', 'Комментарий к адресу',
       'Почтовый индекс', 'Микрорайон', 'Район', 'Город', 'Округ', 'Регион',
       'Страна', 'Часы работы', 'Часовой пояс', 'Рейтинг',
       'Количество отзывов', 'Телефон 1', 'Телефон 2', 'Телефон 3',
       'Веб-сайт 1', 'Веб-сайт 2', 'Twitter', 'ВКонтакте', 'WhatsApp', 'Viber',
       'Telegram', 'YouTube', 'Широта', 'Долгота', '2GIS URL', 'lat', 'lon',
       'link', 'category', 'subcategory', 'new_sub_cat_code', 'subcat_id',
       'h3_9', 'name'],
      dtype='object')

In [63]:
first_file=first_file.append(parks[['lat', 'lon',
       'link', 'category', 'subcategory', 'new_sub_cat_code', 'subcat_id',
       'h3_9', 'name']])

In [64]:
first_file

,index,coordinates,name,link,category,subcategory,subcat_id,lat,lon,h3_9,sub_cat_code_num,new_sub_cat_code,count
0,0.0,"[30.205252, 59.8616]",GPower,https://spb.spravker.ru/agzs/gpower-1.htm,/avto/,/agzs/,3,59.861600,30.205252,8911061503bffff,3.0,agzs or azs,1.0
1,1.0,"[30.205848, 59.86133]",Global Gas,https://spb.spravker.ru/agzs/globalgas.htm,/avto/,/agzs/,3,59.861330,30.205848,8911061503bffff,3.0,agzs or azs,1.0
2,2.0,"[30.246048, 59.877287]",Газпром газомоторное топливо,https://spb.spravker.ru/org/gazprom-transgaz2.htm,/avto/,/agzs/,3,59.877287,30.246048,89110615ec7ffff,3.0,agzs or azs,1.0
3,3.0,"[30.164843, 59.805513]",ВиГаз,https://spb.spravker.ru/agzs/vigaz.htm,/avto/,/agzs/,3,59.805513,30.164843,89110602483ffff,3.0,agzs or azs,1.0
4,4.0,"[30.168601, 59.811463]",РосГаз,https://spb.spravker.ru/org/rosgaz12.htm,/avto/,/agzs/,3,59.811463,30.168601,891106024b3ffff,3.0,agzs or azs,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,NaN,NaN,Павловск,https://2gis.com/firm/70000001057303155,/kultura-i-iskusstvo/,/botanicheskie-sady/,1034,59.688338,30.487851,89110608eb3ffff,NaN,park,NaN
105,NaN,NaN,Сад воды и холмов,https://2gis.com/firm/70000001064591832,/kultura-i-iskusstvo/,/botanicheskie-sady/,1034,60.246545,30.392673,891106a536fffff,NaN,park,NaN
106,NaN,NaN,Парк культуры и отдыха,https://2gis.com/firm/70000001062684743,/kultura-i-iskusstvo/,/botanicheskie-sady/,1034,59.885362,30.985420,8911062957bffff,NaN,park,NaN
107,NaN,NaN,Парк культуры и отдыха,https://2gis.com/firm/70000001062602593,/kultura-i-iskusstvo/,/botanicheskie-sady/,1034,59.887528,30.987161,89110629563ffff,NaN,park,NaN


In [65]:
first_file.to_csv('C:/Users/Asus/Downloads/Telegram Desktop/full_POIs.csv',sep=';',encoding='UTF8')